In [21]:
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from llm import create_agent_llm, get_final_answer,create_llm
from calendar_tools_2 import generate_event_json, get_current_date
from langgraph.prebuilt import create_react_agent


In [2]:

# --- Initializing the scopes and credentials because our agents cannot work without it ---

# Define the scope you need
# IMPORTANT: Use 'calendar.events' to read AND write events
SCOPES = ["https://www.googleapis.com/auth/calendar.events"]

def get_calendar_service():
    """Authenticates and returns a Google Calendar service object."""
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        print("here")
        if creds and creds.expired and creds.refresh_token:
            try:
                creds.refresh(Request())
            except Exception as e:
                print(f"Error refreshing credentials: {e}")
                print(f"we will proceed with a new intiailization of the credentials")
                flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES
            )
                # This is the line that requires human interaction
                print("Please authorize in the browser that opens...")
                creds = flow.run_local_server(port=0)
            # print("here 1")

            # creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES
            )
            # This is the line that requires human interaction
            print("Please authorize in the browser that opens...")
            creds = flow.run_local_server(port=0)
        
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())
            
    return build("calendar", "v3", credentials=creds)

# --- Initialize Service and Agent ---


In [4]:

# 2. Authenticate and get the service object *once*
service = get_calendar_service()
print("Google Calendar service initialized.")

# 3. Create an *instance* of your tool class, passing the service to it
calendar_tool_kit = CalendarTools(service=service)


here
Google Calendar service initialized.


In [3]:

# 4. Get the list of tools *from that instance*
tools = [
    get_current_date,
    generate_event_json
    # calendar_tool_kit.list_google_calendar_events, # if you added more
]


In [4]:
system_prompt = '''
You are a smart, autonomous assistant. Your goal is to help users schedule events in their Google Calendar by **preparing the necessary data**.

You have access to the following tools:

1. `get_current_time()`: 
   Returns the current date, day of the week, and time.

2. `generate_event_json(summary, start_time, end_time, location, description, attendees, timezone)`: 
   Generates the structured JSON object required to create the event. Use this tool immediately once you have calculated all the necessary details.

### Your Rules & Logic

- **CRITICAL DATE RULE:** If the user's request uses a relative date or time (like "today", "this Friday", "next Monday", or "at 3 PM"), you **MUST** use the `get_current_time` tool *first* to get the current date.

- **AGENTIC LOGIC:** Do not guess dates. 
  1. Call `get_current_time` to establish "today."
  2. Calculate the specific ISO 8601 timestamps (e.g., "2025-11-21T15:00:00") for the event start and end.
  3. Call `generate_event_json` with these calculated values.

### DEFAULTS (Apply these if the user does not specify)

- **Duration:** Assume a **1-hour duration**.
- **Location:** Assume "Paris Office".
- **Timezone:** Assume "Europe/Paris".
- **Attendees:** If no specific emails are given, use an empty list `[]`. Do not invent placeholder emails unless the user implies other people are joining without giving addresses.

### FINAL OUTPUT
Do not output raw JSON in your final text response. 
After you have successfully called the `generate_event_json` tool, your final text response to the user should be a clear confirmation that the data is ready for execution, like:
"I have prepared the event details for 'Dentist Appointment' on Friday at 3 PM."
'''

In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

system_prompt_new = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad",optional=True),
    ]
)

#prompt=system_prompt_new.invoke({"input":"what's the date of today ?" }) #"Create an event for 'Dentist Appointment' this Friday at 3 PM."
prompt=system_prompt_new.invoke({"Create an event for 'Dentist Appointment' this Friday at 3 PM."})

In [22]:
system_prompt = """
You are a strict calendar automation assistant. 
Your ONLY goal is to generate a JSON object for the user.

### TOOL USAGE PROTOCOL
1. **Check Date:** You MUST call `get_current_time` first if the request contains relative dates (today, next Friday).
2. **Calculate:** Compute the specific ISO 8601 timestamps.
3. **Generate:** Call `generate_event_json` with the computed details.

### CRITICAL OUTPUT RULES
- **DO NOT** output internal function tags like `<|python_tag|>` or `<function...`. Use the provided tools natively.
- **DO NOT** talk to the user. Do not say "Here is the JSON" or "I have scheduled it".
- **FINAL ANSWER:** After the `generate_event_json` tool runs and returns the data, your final response to the user must be **ONLY the JSON object** returned by the tool. 
- Output the raw JSON string only. No Markdown formatting (like ```json).

### DEFAULTS
- Duration: 1 hour.
- Location: "Paris Office".
- Timezone: "Europe/Paris".
- Attendees: [] (Empty list).
"""

# 5. Now, build your agent with this list of tools
agent= create_agent_llm(
    tools=tools,
    system_prompt=system_prompt)

print("Agent is ready.")



Agent is ready.


In [23]:

# --- 5. Run & Stream ---
print("--- STARTING AGENT ---")

inputs = {"messages": [("user", "Create a dentist appointment for this Friday at 3pm")]}

# stream_mode="values" gives you the full list of messages at every step
# This allows you to see the System -> User -> AI -> Tool -> AI sequence perfectly
# for chunk in agent.stream(inputs, stream_mode="update"):
#     messages = chunk["messages"]
#     last_msg = messages[-1]
response = agent.invoke(inputs)

--- STARTING AGENT ---


In [25]:
get_final_answer(response)

'<|python_tag|><function/get_current_date>{}; <function/get_current_date>{}; <function coronary="generate_event_json">{"summary": "Dentist Appointment", "start_time": "2025-11-21T15:00:00", "end_time": "2025-11-21T16:00:00", "location": "Paris Office", "description": "", "attendees": [], "timezone": "Europe/Paris"}[/function]'

In [8]:
# inputs = {
#     "messages": [
#         ("human", "Create an event for 'Dentist Appointment' this Friday at 3 PM.")
#     ]
# }
for chunk in agent.stream(prompt, stream_mode="update"):
    print(chunk)




# response = agent.invoke(inputs)
# final_answer=get_final_answer(response)
# print(final_answer)
# print(response['output'])

In [17]:
print("--- DEBUGGING STEPS ---")

for chunk in agent.stream(prompt):
    # The chunk here usually contains 'actions' and 'messages'
    
    if "actions" in chunk:
        for action in chunk["actions"]:
            print(f"\n🤖 AGENT IS CALLING: {action.tool}")
            print(f"   Tool Input: {action.tool_input}")
            
    if "steps" in chunk:
        for step in chunk["steps"]:
            # This is the result of the tool
            print(f"\n📦 TOOL RESULT: {step.observation}")
            
    if "output" in chunk:
        print(f"\n🏁 FINAL ANSWER: {chunk['output']}")

--- DEBUGGING STEPS ---


KeyboardInterrupt: 

In [8]:
response

NameError: name 'response' is not defined